# Preprocesamiento 

In [10]:
import os

def obtener_ultimo_archivo_creado(directorio):
    archivos = [os.path.join(directorio, f) for f in os.listdir(directorio) 
                if os.path.isfile(os.path.join(directorio, f))]
    
    if not archivos:
        return None
    
    ultimo_archivo = max(archivos, key=os.path.getctime)
    return ultimo_archivo

# Ejemplo de uso:
directorio = "/home/ares/iqoptions_bot/data_csv"
ultimo = obtener_ultimo_archivo_creado(directorio)

if ultimo:
    print("El último archivo creado es:", ultimo)
else:
    print("No se encontraron archivos en la carpeta.")


El último archivo creado es: /home/ares/iqoptions_bot/data_csv/2025-04-29_13-23.csv


In [11]:
import pandas as pd 

df = pd.read_csv(f'{ultimo}')

df.head(2)

,from_dt,to_dt,open,close,min,max,volume
0,2025-04-24 07:02:30,2025-04-24 07:03:00,1.13579,1.135935,1.135785,1.136105,379
1,2025-04-24 07:03:00,2025-04-24 07:03:30,1.13594,1.135795,1.135710,1.135990,281


In [14]:
import pandas as pd
from ta.momentum import RSIIndicator, StochasticOscillator
from ta.volatility import AverageTrueRange
from sklearn.preprocessing import StandardScaler

def calcular_indicadores_tecnicos(df):
    """
    Calcula indicadores técnicos y los estandariza.

    Parámetros:
        df (pd.DataFrame): Debe contener columnas ['open', 'high', 'low', 'close', 'volume']

    Retorna:
        pd.DataFrame: DataFrame original con columnas de indicadores añadidas y estandarizadas.
    """

    # Validación de columnas necesarias
    columnas_requeridas = ['open', 'max', 'min', 'close', 'volume']
    for col in columnas_requeridas:
        if col not in df.columns:
            raise ValueError(f"Falta la columna requerida: {col}")

    df = df.copy()

    # RSI
    df['rsi_5'] = RSIIndicator(close=df['close'], window=5).rsi()
    df['rsi_7'] = RSIIndicator(close=df['close'], window=7).rsi()

    # Estocástico
    stoch = StochasticOscillator(high=df['max'], low=df['min'], close=df['close'], window=5, smooth_window=3)
    df['stoch_k'] = stoch.stoch()
    df['stoch_d'] = stoch.stoch_signal()

    # ATR
    df['atr_5'] = AverageTrueRange(high=df['max'], low=df['min'], close=df['close'], window=5).average_true_range()

    # Velocidad del precio
    df['price_speed_3'] = df['close'].pct_change(periods=3)
    df['price_speed_5'] = df['close'].pct_change(periods=5)

    # Columnas a estandarizar
    columnas_indicadores = ['rsi_5', 'rsi_7', 'stoch_k', 'stoch_d', 'atr_5', 'price_speed_3', 'price_speed_5']
    return df
    # Estandarización
    # scaler = StandardScaler()
    # df[columnas_indicadores] = scaler.fit_transform(df[columnas_indicadores])

    


In [16]:
ind = calcular_indicadores_tecnicos(df)
ind.head(12)

,from_dt,to_dt,open,close,min,max,volume,rsi_5,rsi_7,stoch_k,stoch_d,atr_5,price_speed_3,price_speed_5
0,2025-04-24 07:02:30,2025-04-24 07:03:00,1.135790,1.135935,1.135785,1.136105,379,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
1,2025-04-24 07:03:00,2025-04-24 07:03:30,1.135940,1.135795,1.135710,1.135990,281,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
2,2025-04-24 07:03:30,2025-04-24 07:04:00,1.135790,1.135695,1.135695,1.135815,202,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
3,2025-04-24 07:04:00,2025-04-24 07:04:30,1.135700,1.135785,1.135700,1.135900,319,NaN,NaN,NaN,NaN,0.000000,-0.000132,NaN
4,2025-04-24 07:04:30,2025-04-24 07:05:00,1.135780,1.135945,1.135765,1.135970,224,63.098346,NaN,60.975610,NaN,0.000226,0.000132,NaN
5,2025-04-24 07:05:00,2025-04-24 07:05:30,1.135950,1.135705,1.135680,1.136205,922,34.747184,NaN,4.761905,NaN,0.000286,0.000009,-0.000202
6,2025-04-24 07:05:30,2025-04-24 07:06:00,1.135700,1.135745,1.135675,1.135920,475,40.332521,39.768053,13.207547,26.315021,0.000278,-0.000035,-0.000044
7,2025-04-24 07:06:00,2025-04-24 07:06:30,1.135740,1.135840,1.135615,1.135895,368,52.422495,50.045874,38.135593,18.701682,0.000278,-0.000092,0.000128
8,2025-04-24 07:06:30,2025-04-24 07:07:00,1.135845,1.135780,1.135765,1.135950,375,45.193164,44.456264,27.966102,26.436414,0.000259,0.000066,-0.000004
9,2025-04-24 07:07:00,2025-04-24 07:07:30,1.135785,1.135680,1.135555,1.135790,442,35.106876,36.524150,19.230769,28.444155,0.000255,-0.000057,-0.000233


In [18]:
ind.isnull().sum()

from_dt           0
to_dt             0
open              0
close             0
min               0
max               0
volume            0
rsi_5             4
rsi_7             6
stoch_k          19
stoch_d          27
atr_5             0
price_speed_3     3
price_speed_5     5
dtype: int64

In [19]:
ind = ind.dropna()
ind.isnull().sum()

from_dt          0
to_dt            0
open             0
close            0
min              0
max              0
volume           0
rsi_5            0
rsi_7            0
stoch_k          0
stoch_d          0
atr_5            0
price_speed_3    0
price_speed_5    0
dtype: int64

In [21]:
ind.shape

(9973, 14)